In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

import itertools

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

In [3]:
data=pd.read_csv("../data/raw/databinarystudents.csv").head(150)

## Entrenamiento del modelo

In [4]:
roc = []
accur = []

from sklearn.linear_model import RidgeClassifier

for j in range(1+1000,101+1000):
    
    train, test = train_test_split(data, test_size = 0.2, random_state=j)
    #print(train.shape)
    train=train.drop(['id'], axis=1)
    X_train=train.drop(['yL'], axis=1)
    Y_train=train['yL']
    test=test.drop(['id'], axis=1)
    X_test=test.drop(['yL'], axis=1)
    Y_test=test['yL']
    
    sel= ['x1','x17','x18','x20','x23','x24','x4']
    #sel= ['x17', 'x18', 'x23', 'x4', 'x10', 'x24']
    num_features = sel
    num_pipeline = Pipeline([
        ('scaler',StandardScaler()),
        ('imputer',SimpleImputer(strategy='median'))
        ])
    col_transformer = ColumnTransformer(transformers=[
        ('num', num_pipeline, num_features)
        ])
    
    columns=X_train.columns.values.tolist()
    col_transformer.fit(X_train.fillna(0))
    X_train[num_features]=col_transformer.transform(X_train.fillna(0))
    X_test[num_features]=col_transformer.transform(X_test.fillna(0))
    
    # Modelo con características seleccionadas
    
    final_m = RidgeClassifier()
    
    
    final_m.fit(X_train.filter(sel, axis=1), Y_train)
    
    #y_pred_test = final_m.predict_proba()[:,1]
    d = final_m.decision_function(X_test.filter(sel, axis=1))
    
    y_pred_test = np.exp(d) / (1 + np.exp(d))
    
    ROC_AUC_final = roc_auc_score(Y_test, y_pred_test)
    
    y_pred_test1 = final_m.predict(X_test.filter(sel, axis=1))
    acc = accuracy_score(Y_test, y_pred_test1)
    
    roc.append(ROC_AUC_final)
    accur.append(acc)

#### AUC y Accuracy

In [5]:
summar1 = pd.concat([pd.DataFrame(roc), pd.DataFrame(accur)],axis=1)

In [6]:
summar1.columns=['ROC_AUC','Accuracy']

Distribución del AUC

In [8]:
print('Q1:', np.percentile(summar1['ROC_AUC'],25))
print('Median:',np.median(summar1['ROC_AUC']))
print('Q3:', np.percentile(summar1['ROC_AUC'],75))

Q1: 0.7361111111111112
Median: 0.7909276018099548
Q3: 0.8337053571428572


Distribución accuracy

In [10]:
print('Q1:', np.percentile(summar1['Accuracy'],25))
print('Median:',np.median(summar1['Accuracy']))
print('Q3:', np.percentile(summar1['Accuracy'],75))

Q1: 0.6333333333333333
Median: 0.7
Q3: 0.7333333333333333
